In [1]:
from MEC import Circle, Point, welzl

def calculate_minimum_enclosing_circle(points):
    """
    Calculate the minimum enclosing circle for a set of points using Welzl's algorithm.
    Returns the center and radius of the circle.
    """
    mec = welzl(points)  
    center=[]
    center.append(mec.C.X)
    center.append(mec.C.Y)
    return center, mec.R

In [2]:
from utils import calculate_euclidean_distance, calculate_real_width
from skeletonization import skeletonize_mask,create_filled_binary_mask, skeletonize_mask, find_longest_path
import cv2
import fiftyone as fo


import numpy as np

def process_segmentations(segmentation_path):
    """
    Process the segmentations from the TXT file, calculate the minimum enclosing circle for each prawn.
    """
    segmentations = []
    skeletons=[]
    hulls=[]
    skeletons_straight=[]
    skeletons_straight_2=[]
    seg_closeds=[]
    skeletons_2=[]
    box_diagonal=[] 
    boxes=[]
    masks=[]






    
    # Open the segmentation file and process each line
    with open(segmentation_path, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))
            binary_mask = create_filled_binary_mask(coords, 640, 640)
            
            
            binary_mask_no= create_filled_binary_mask(coords, 640, 640,gaussian_blur=False) 

            binary_dilated = cv2.dilate(binary_mask_no, np.ones((15, 15), np.uint8), iterations=1)     


            #contour dilated
            contures_dil, _ = cv2.findContours(binary_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    

            prawn_conture_dil = max(contures_dil, key=cv2.contourArea)

            coords_contour_dil = np.column_stack(prawn_conture_dil).flatten()

            normalized_coords_bin=[(coords_contour_dil[i]/640, coords_contour_dil[i+1]/640) for i in range(0, len(coords_contour_dil), 2)]  # Extract points (x, y)
            # coords_bin = np.column_stack(np.nonzero(binary_dilated)).flatten()

            # normalized_coords_bin=[(coords_bin[i+1]/640, coords_bin[i]/640) for i in range(0, len(coords_bin), 2)]  # Extract points (x, y)



            masks.append(fo.Polyline(
                points=[normalized_coords_bin],
                closed=True,
                filled=False,
            ))



            # #convert binary mask to normalized coordinates
            # binary_mask_smooth = binary_mask.astype(np.uint8)
            # #x,y coordinates of the mask
            # coords_bin = np.column_stack(np.nonzero(binary_mask_smooth)).flatten()

            # normalized_coords_bin=[(coords_bin[i+1]/640, coords_bin[i]/640) for i in range(0, len(coords_bin), 2)]  # Extract points (x, y)




        
            # #thin the mask
            # thinned=skeletonize_mask(binary_mask)

            # # skeleton = skeletonize_mask(binary_mask)
            # skeleton = thinned
            # skeleton_coords = np.column_stack(np.nonzero(skeleton))
            # normalized_coords,max_length = find_longest_path(skeleton_coords,(640,640),(2988,5312))

            # normalized_coords = [(x, y) for y, x in normalized_coords]  # Convert to (y, x) format

            # #only the first and last points of the skeleton
            # normalized_coords_straight = [normalized_coords[0], normalized_coords[-1]]  

            
            thinned_2=skeletonize_mask(binary_mask_no)

            # skeleton = skeletonize_mask(binary_mask)
            skeleton_2 = thinned_2
            skeleton_coords_2 = np.column_stack(np.nonzero(skeleton_2))
            normalized_coords_2,max_length_2 = find_longest_path(skeleton_coords_2,(640,640),(2988,5312))

            normalized_coords_2 = [(x, y) for y, x in normalized_coords_2]  # Convert to (y, x) format

            #only the first and last points of the skeleton
            normalized_coords_straight_2 = [normalized_coords_2[0], normalized_coords_2[-1]]  






            #convex hull diameter
            contures, _ = cv2.findContours(binary_mask_no, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            
            prawn_conture = max(contures, key=cv2.contourArea)  

            # Compute the minimum area rectangle enclosing the shrimp
            rect = cv2.minAreaRect(prawn_conture)
            box_points = cv2.boxPoints(rect)
            box_points = np.int0(box_points)

            original_size = (640, 640)
            new_size = (5312, 2988)

            # Scaling factors
            scale_x = new_size[0] / original_size[0]  # 5312 / 640
            scale_y = new_size[1] / original_size[1]  # 2988 / 640

            box_points_scaled = np.array([(point[0] * scale_x, point[1] * scale_y) for point in box_points])

            width= calculate_euclidean_distance(box_points_scaled[0], box_points_scaled[1])
            height= calculate_euclidean_distance(box_points_scaled[1], box_points_scaled[2])

            max_length_box=max(width,height)


           

            normalized_bounding_box = [(box_points[i][0]/640, box_points[i][1]/640) for i in range(0, len(box_points))] 
            
            # Extract points (x, y) 
            box=fo.Polyline(
                points=[normalized_bounding_box],
                closed=True,
                filled=False,
                max_length=max_length_box
            )


            boxes.append(box)


            hull_points = cv2.convexHull(prawn_conture, returnPoints=True)


            
# Scaling factors to convert from 640x640 to 5312x2988
            scale_x = 5312 / 640
            scale_y = 2988 / 640

        # Scale the points to the new resolution
            scaled_hull_points = []
            for point in hull_points:
                x, y = point[0]
                scaled_x = x * scale_x
                scaled_y = y * scale_y
                scaled_hull_points.append([scaled_x, scaled_y])

            # Convert to numpy array for easier handling
            scaled_hull_points = np.array(scaled_hull_points, dtype=np.float32)

            # Now, find the maximum Euclidean distance (convex hull diameter) using the scaled points
            max_distance = 0
            point1 = None
            point2 = None

            # Loop through all pairs of scaled points to find the maximum distance
            for i in range(len(scaled_hull_points)):
                for j in range(i + 1, len(scaled_hull_points)):
                    distance = calculate_euclidean_distance(scaled_hull_points[i], scaled_hull_points[j])
                    if distance > max_distance:
                        max_distance = distance
                        point1 = scaled_hull_points[i]
                        point2 = scaled_hull_points[j]

            # The result is max_distance (in pixels) in the 5312x2988 image


            normalzied_points_hull = [(point1[0]/5312, point1[1]/2988), (point2[0]/5312, point2[1]/2988)]  # Extract points (x, y)

            hull=fo.Polyline(
                points=[normalzied_points_hull],
                closed=False,
                filled=False,
                max_length=max_distance
            )

            # skeleton_straight=fo.Polyline(
            #     points=[normalized_coords_straight],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )
            # skeletons_straight.append(skeleton_straight)

            skeleton_straight_2=fo.Polyline(
                points=[normalized_coords_straight_2],
                closed=False,
                filled=False,
                max_length=max_length_2,
                
            )
            skeletons_straight_2.append(skeleton_straight_2)




            hulls.append(hull)

            # skeleton=fo.Polyline(
            #     points=[normalized_coords],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )

            # skeletons.append(skeleton)
            
            skeleton_2=fo.Polyline( 
                points=[normalized_coords_2],
                closed=False,
                filled=False,
                max_length=max_length_2)
            skeletons_2.append(skeleton_2)
              # Convert the line to a list of floats
            normalzied_points = [(coords[i]/640, coords[i + 1]/640) for i in range(0, len(coords), 2)]  # Extract points (x, y)
            points = [Point(x*5312, y*2988) for x, y in normalzied_points] 
            

            
             # Convert to Point objects    
            # Calculate the minimum enclosing circle (center and radius)
            center, radius = calculate_minimum_enclosing_circle(points)
            diameter = radius * 2

            segmentation = fo.Polyline(
                points=[normalzied_points],
                closed=True,
                filled=False,
                diameter=diameter,
                center=center,
                max_length_skeleton=max_length_2,
                max_length_hull=max_distance,
                max_length_box=max_length_box
            )

            #smooth segmentation  wirh closing
            # seg_closed=fo.Polyline(
            #     points=[normalized_coords_bin],
            #     closed=True,
            #     filled=False,
            #     max_length=max_length
            # )

            # seg_closeds.append(seg_closed)                


            segmentations.append(segmentation)







                     # Store the segmentation information (center, radius, and diameter)

    return segmentations,skeletons,hulls, skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks

In [6]:
import fiftyone.core.labels as fol
from tqdm import tqdm
import fiftyone as fo
import os

def process_images(image_paths, prediction_folder_path, dataset):
    print("Processing images...")
    
    """
    Processes images by matching segmentation with bounding boxes and calculating prawn sizes.
    """
    for image_path in tqdm(image_paths):
        # filename = os.path.splitext(os.path.basename(image_path))[0]
        

           
        # prediction_txt_path = os.path.join(prediction_folder_path, f"{os.path.basename(image_path).split('.')[0]}_segmentations.txt")

        core_name = os.path.splitext(os.path.basename(image_path))[0]

        # Construct the path to the corresponding segmentation file
        prediction_txt_path = os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")

        # core_name=filename.split('.')[0]
        # # Construct the path to the prediction (segmentation) file
        # prediction_txt_path = os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")
        # if not os.path.exists(prediction_txt_path):
        #     print(f"No segmentation file found for {filename}")
        #     continue


        # Parse the segmentations to get the minimum enclosing circles
        segmentations,skeletons,hulls,skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks = process_segmentations(prediction_txt_path)

        # Save the modified image (with circles drawn)

        # Create a new sample for FiftyOne
        sample = fo.Sample(filepath=image_path)

        # Iterate over each bounding box in the filtered data
        sample["segmentations"] = fol.Polylines(polylines=segmentations)

        # sample["skeletons"] = fol.Polylines(polylines=skeletons)

        sample["hulls"] = fol.Polylines(polylines=hulls)    

        # sample["skeletons_straight"] = fol.Polylines(polylines=skeletons_straight)

        # sample['seg_closeds']=fol.Polylines(polylines=seg_closeds)
        
        sample['skeletons_no_smooth']=fol.Polylines(polylines=skeletons_2)

        sample["skeletons_straight_no_smooth"] = fol.Polylines(polylines=skeletons_straight_2)

        sample['boxes']=fol.Polylines(polylines=boxes)

        sample['masks']=fol.Polylines(polylines=masks)
        # Add the processed sample to the FiftyOne dataset
        dataset.add_sample(sample)

In [25]:
import math
def process_detection_by_circle(segmentation):
    """
    Process the prawn detection based on the enclosing circle's diameter.
    Update the filtered dataframe with the real-world size of the prawn.
    """

    # Fetch height in mm and other metadata
    height_mm =500
    #focal length based on pond type
    
    focal_length = 24.72


    # focal_length = 24.22  # Camera focal length
    pixel_size = 0.00716844  # Pixel size in mm

    poly=segmentation

    fov=75
    FOV_width=2*height_mm*math.tan(math.radians(fov/2))


    # Get the diameter of the circle in pixels
    predicted_diameter_pixels = poly['diameter']


    predicted_skeleton_length=poly['max_length_skeleton']  

    predicted_hull_length=poly['max_length_hull']

     
    predicted_box_length=poly['max_length_box']

    # Calculate the real-world prawn size using the box
    real_length_mm_box = calculate_real_width(focal_length, height_mm, predicted_box_length, pixel_size) 


    hull_length_cm = calculate_real_width(focal_length, height_mm, predicted_hull_length, pixel_size)    

    # Calculate the real-world prawn size using the enclosing circle's diameter
    real_length_cm = calculate_real_width(focal_length, height_mm, predicted_diameter_pixels, pixel_size)

    ske_length_cm = calculate_real_width(focal_length, height_mm, predicted_skeleton_length, pixel_size)    


    hull_length_fov=FOV_width*predicted_hull_length/5312
    diameter_length_fov=FOV_width*predicted_diameter_pixels/5312
    skeleton_length_fov=FOV_width*predicted_skeleton_length/5312

    box_length_fov=FOV_width*predicted_box_length/5312

    true_length=143
    error_percentage_MEC_fov = abs(diameter_length_fov - true_length) / true_length * 100

    error_percentage_hull_fov = abs(hull_length_fov - true_length) / true_length * 100

    error_percentage_skeleton_fov = abs(skeleton_length_fov - true_length) / true_length * 100  

    error_percentage = abs(real_length_cm - true_length) / true_length * 100

    error_percentage_skeleton = abs(ske_length_cm - true_length) / true_length * 100    

    error_percentage_hull = abs(hull_length_cm - true_length) / true_length * 100

    error_percentage_box_fov = abs(box_length_fov - true_length) / true_length * 100

    closest_detection_label = f'true length: {true_length:.2f}mm, MPError: {error_percentage_hull_fov:.2f}%, , pred length: {hull_length_fov:.2f}mm ,error percentage skeleton: {error_percentage_skeleton_fov:.2f}%, , pred length: {skeleton_length_fov:.2f}cm, error percentage box: {error_percentage_box_fov:.2f}%, pred length: {box_length_fov:.2f}mm, '
    poly.label = closest_detection_label


In [26]:
molt_image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted"
molt_prediction=r'C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized'

import fiftyone as fo
dataset = fo.Dataset("molt", overwrite=True)

# Load the dataset
image_paths = [os.path.join(molt_image_path, image) for image in os.listdir(molt_image_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
prediction_paths_text = [os.path.join(molt_prediction, txt) for txt in os.listdir(molt_prediction) if txt.endswith('.txt')]

# Process images

process_images(image_paths, molt_prediction, dataset)

# Process segmentations
for sample in dataset:
    for segmentation in sample["segmentations"]['polylines']:
        
        process_detection_by_circle(segmentation)

# Launch FiftyOne session
session = fo.launch_app(dataset)

Processing images...


  0%|          | 0/6 [00:00<?, ?it/s]

Processing image: undistorted_vlcsnap-2024-09-19-21h46m36s160


C:\Users\gbo10\AppData\Local\Temp\ipykernel_44812\2001072356.py:115: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)
 33%|███▎      | 2/6 [00:00<00:00,  5.01it/s]

Processing image: undistorted_vlcsnap-2024-09-19-21h46m54s892
Processing image: undistorted_vlcsnap-2024-09-19-21h47m01s074


...

Processing image: undistorted_vlcsnap-2024-09-19-21h47m15s479


 83%|████████▎ | 5/6 [00:00<00:00,  5.64it/s]

Processing image: undistorted_vlcsnap-2024-09-19-21h49m34s599
Processing image: undistorted_vlcsnap-2024-09-19-21h49m47s447


100%|██████████| 6/6 [00:01<00:00,  5.92it/s]


KeyError: 'Polyline'